# Using local db and csv dump to load Clinical Trials data


In [1]:
# Imports
from tqdm import tqdm
from datetime import datetime as dt




# Accessing data from the local csv file

In [2]:
import pandas as pd

# wonder if we can read it in into memory:P - YES we can.
print(f"{dt.now()}: Reading in csv data...")
alltrials_df = pd.read_csv('../data/alltrials.csv', index_col=False, on_bad_lines='warn', sep="\t")
print(f"{dt.now()}: Done reading in csv data...")

2023-10-18 17:08:52.290302: Reading in csv data...


In [ ]:
alltrials_df

,Unnamed: 0,NCTId,Condition,BriefTitle
0,0.0,NCT00509938,"['Hematopoietic Stem Cell Transplantation', 'B...",Safety of a Single Dose of 5 mg of hLF1-11 Giv...
1,1.0,NCT00501553,['Heart Failure'],Vitamin D in Patients With Heart Failure
2,2.0,NCT00508131,"['Anemia', 'Iron Deficiency', 'Stunting']",Effectiveness of Iron-Fortified Milk on Iron S...
3,3.0,NCT00505570,"['Migraine', 'Migraine With Aura', 'Patent For...",PRIMA PFO Migraine Trial
4,4.0,NCT00500214,"['Postoperative Pain', 'Postoperative Hyposthe...",Ilioinguinal Nerve Excision in Open Mesh Repai...
...,...,...,...,...
101002,99995.0,NCT05903859,"['Infertility, Male']",Improvement of Reproductive Function in Men Wi...
101003,99996.0,NCT05902767,"['Stroke', 'Dementia']",Nut Supplementation to Mitigate Post-stroke Co...
101004,99997.0,NCT05907148,['Parkinson Disease'],Effects of Sensory Integration Training on Bal...
101005,99998.0,NCT05906706,['Bullous Pemphigoid'],Compassionate Use of Dupilumab for Adult Patie...


# Accessing data from the local sqlite database

In [1]:
import pandas as pd
import sqlite3
print(f"{dt.now()}: Reading in data from db...")
conn = sqlite3.connect('../data/alltrials.db')  # Path to the db file
table_name = 'alltrials' # Name of the table to read
# In inital run I was not sure what goes into the 'table_name' and was putting the path '../data/alltrials' instead of just 'alltrials'. 
# Use the path verison f you generated the db with the older version of the notebook.
trials_query = f"""SELECT * FROM {table_name} LIMIT 10"""
result = conn.execute(trials_query)
alltrials_df = pd.DataFrame.from_records(result)

print(f"{dt.now()}: Done reading in data from db...")

NameError: name 'conn' is not defined

In [ ]:
alltrials_df = pd.DataFrame.from_records(result)


# Accessing data from the local postgres database
Following the guidelinas to create the database form here: 
https://aact.ctti-clinicaltrials.org/snapshots

After downloading the most recent DB from here:
https://aact.ctti-clinicaltrials.org/download 

With a little bit of help from chatGPT.

It requires switching to the postgres user and installing the postgres server.

It requires setting a new password inside postgres for postgrwes user.

host and port are set to defaults.

The created db here was named 'aact' and the table schema is 'ctgov'.

In [15]:
import psycopg2

# Set your connection parameters
db_params = {
    'dbname': 'aact',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': 5432  # Default PostgreSQL port
}

# Connect to the database
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("Connected to the database!")
    
    # You can now execute SQL queries
    cursor.execute("SELECT * FROM ctgov.conditions LIMIT 10") # What other tables are there? See cell below.
    result = cursor.fetchall()
    print(result)
  
    # Define the column names
    column_names = [desc[0] for desc in cursor.description]

    # Create a Pandas DataFrame
    df = pd.DataFrame(result, columns=column_names)
    print(df)


    # Don't forget to close the cursor and connection when done
    cursor.close()
    conn.close()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error:", error)


Connected to the database!
[(63298469, 'NCT06021340', 'IVF', 'ivf'), (63298574, 'NCT06012656', 'Rheumatoid Arthritis', 'rheumatoid arthritis'), (63298575, 'NCT06012656', 'Congenital Hip Dysplasia', 'congenital hip dysplasia'), (63298579, 'NCT06010732', 'Caries', 'caries'), (63298667, 'NCT05811728', 'Low Level Laser Therapy', 'low level laser therapy'), (63298668, 'NCT05811728', 'Canine Retraction', 'canine retraction'), (63298669, 'NCT05811728', 'Orthodontic Tooth Movement', 'orthodontic tooth movement'), (63298671, 'NCT05799976', 'Diabetes Mellitus', 'diabetes mellitus'), (63298845, 'NCT05468970', 'Bone and Soft Tissue Tumors', 'bone and soft tissue tumors'), (63298846, 'NCT05468970', 'Clinical Features', 'clinical features')]
         id       nct_id                         name  \
0  63298469  NCT06021340                          IVF   
1  63298574  NCT06012656         Rheumatoid Arthritis   
2  63298575  NCT06012656     Congenital Hip Dysplasia   
3  63298579  NCT06010732          

In [22]:
# What are the tables in the database??
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_params)

    # Create a cursor
    cursor = connection.cursor()

    # SQL query to list all tables
    query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'ctgov'"

    # Execute the query
    cursor.execute(query)

    # Fetch all table names
    table_names = [row[0] for row in cursor.fetchall()]
    # Print the table names
    for table in table_names:
        print(table)

    # Close the cursor and the database connection
    cursor.close()
    connection.close()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

browse_interventions
all_browse_conditions
conditions
all_browse_interventions
id_information
all_cities
countries
all_conditions
design_groups
all_countries
facilities
all_design_outcomes
all_facilities
interventions
all_group_types
keywords
all_id_information
design_outcomes
all_intervention_types
all_interventions
browse_conditions
all_keywords
sponsors
all_overall_official_affiliations
all_overall_officials
all_primary_outcome_measures
all_secondary_outcome_measures
all_sponsors
all_states
overall_officials
brief_summaries
baseline_counts
baseline_measurements
categories
design_group_interventions
detailed_descriptions
eligibilities
intervention_other_names
calculated_values
covid_19_studies
designs
studies
central_contacts
search_results
drop_withdrawals
documents
facility_investigators
milestones
mesh_headings
mesh_terms
outcome_analyses
outcomes
outcome_counts
outcome_measurements
outcome_analysis_groups
ipd_information_types
links
provided_documents
reported_event_totals
report